In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from import_casa import casa, cano

In [3]:
TARGET_DIR = "20210406"
result_dir = casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}"
json_paths = [x for x in result_dir.iterdir() if x.suffix==".json"]

In [4]:
with json_paths[1].open("r", encoding="UTF-8") as fin:
    annots = json.load(fin)

In [5]:
list(annots[0].keys())

['completions', 'data', 'id']

In [6]:
len(annots)

382

In [7]:
aspect_list = []

In [8]:
for annot_i, annot_x in tqdm(enumerate(annots)):
    ctxs = [x for x in annot_x["completions"][0]["result"]  if x.get("from_name", "").startswith("ctx-sentiment")]
    if ctxs:
        print(annot_i)
        aspects = cano.process_thread_annotations(annot_x)
        break
        # aspect_list.extend(aspects)

1it [00:00, ?it/s]

1


In [8]:
def find_thread_idx(annots, thread_idx):
    iter_annot = filter(lambda x: x["data"]["thread_idx"]==thread_idx, annots)
    return list(iter_annot)[0]

In [13]:
annots = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        obj = json.load(fin)
    annots.extend(obj)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.22it/s]


In [30]:
annot_x = find_thread_idx(annots, 220)

In [34]:
aspects, dbg = cano.process_thread_annotations(annot_x, True)

In [35]:
aspects

[<AnnotAspect (std), 1-5, Thread 220 ://優惠不錯 />,
 <AnnotAspect (std), 1-5, Thread 220 ://比較便宜/4>,
 <AnnotAspect (std), 1-5, Thread 220 :臺灣大哥大/[通訊]網速/被靠北到死 /1>,
 <AnnotAspect (std), 1-5, Thread 220 :臺灣大哥大/[資費]續約攜碼/最好/5>,
 <AnnotAspect (std), 1-5, Thread 220 :遠傳電信/[資費]續約攜碼/比較划算欸，比另外兩家都更便宜 些/4>,
 <AnnotAspect (std), 1-5, Thread 220 :遠傳電信/[資費]續約攜碼/還蠻推/4>]

In [36]:
dbg

{'relations': {'Z6okmKmpxZ': ['DDXHqKpP4C', 'u9uFQUkcng'],
  'DDXHqKpP4C': ['Z6okmKmpxZ'],
  'u9uFQUkcng': ['Z6okmKmpxZ'],
  '3N510DFllY': ['qKOI3WDVGa', 'Q1BJ1nYv2c'],
  'qKOI3WDVGa': ['3N510DFllY'],
  'Q1BJ1nYv2c': ['3N510DFllY'],
  'X-zotPemPn': ['xuUXa83jFo', 'xSo_Pep-3L'],
  'xuUXa83jFo': ['X-zotPemPn'],
  'xSo_Pep-3L': ['X-zotPemPn'],
  'ipU_I4Up-Q': ['2UkhZzUG81', 'dS1luUa09L'],
  '2UkhZzUG81': ['ipU_I4Up-Q'],
  'dS1luUa09L': ['ipU_I4Up-Q'],
  '0bHE5Ow7Uz': ['C8OHMfbLL_', 'FVcq_RMHH4'],
  'C8OHMfbLL_': ['0bHE5Ow7Uz'],
  'FVcq_RMHH4': ['0bHE5Ow7Uz'],
  'fFJflhBp-l': ['b_GNGShOtQ', 'iEqwtM8KjW'],
  'b_GNGShOtQ': ['fFJflhBp-l'],
  'iEqwtM8KjW': ['fFJflhBp-l']},
 'annot_spans': {'Anq2I42TSQ': [<AnnotSpan [Anq2I42TSQ]: ent-norm, 遠傳電信>],
  'X-zotPemPn': [<AnnotSpan [X-zotPemPn]: ent-norm, 臺灣大哥大>],
  'ipU_I4Up-Q': [<AnnotSpan [ipU_I4Up-Q]: ent-norm, 臺灣大哥大>],
  '0bHE5Ow7Uz': [<AnnotSpan [0bHE5Ow7Uz]: ent-norm, 遠傳電信>],
  'xuUXa83jFo': [<AnnotSpan [xuUXa83jFo]: attr-norm, [通訊]網速>],
  '2Uk

In [25]:
aspects

[<AnnotAspect (std), 6-75, Thread 4863 :中華電信/[通訊]頻段/很期待/5>]

In [44]:
aspect_list = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        annots = json.load(fin)
    for annot_i, annot_x in enumerate(annots):        
        aspects = cano.process_thread_annotations(annot_x)        
        aspect_list.extend(aspects)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 41.43it/s]

安平區
台北市區
499之亂後
NP
日本國際漫遊
成功嶺
海景


In [45]:
len(aspect_list)

1401

In [46]:
import pandas as pd
data_items = []
for aspect_x in aspect_list:
    batch_idx = aspect_x.batch_idx
    thread_idx = aspect_x.thread_idx
    serial = aspect_x.serial
    aspect_tuple = aspect_x.make_tuple()
    memo = aspect_x.memo
    ent_rawtext = aspect_x.raw_text(cano.AspectEnum.Entity)
    attr_rawtext = aspect_x.raw_text(cano.AspectEnum.Attribute)
    is_context = aspect_x.has_context_only
    if all(not x.strip() for x in aspect_tuple[0:3]):
        continue
    data_items.append((batch_idx, serial, thread_idx, is_context,
                      *aspect_tuple, ent_rawtext, attr_rawtext))
    

In [50]:
aspect_df = pd.DataFrame(data_items, 
             columns=["batch_idx", "serial", "thread_idx", "is_context", "ent_norm", "attr_norm", "evaltext", 
                      "rating", "ent_rawtext", "attr_rawtext"])
aspect_df.to_csv(result_dir/f"aspect_tuples_{TARGET_DIR}.2.csv", encoding="UTF-8", index=False)

In [51]:
aspect_df

,batch_idx,serial,thread_idx,is_context,ent_norm,attr_norm,evaltext,rating,ent_rawtext,attr_rawtext
0,0,43,3260,True,臺灣之星,[通訊]頻段,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,1,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
1,0,7,281,False,中華電信,[通訊]國內電信漫遊,很可以,4,中華,訊號
2,0,13,932,False,臺灣之星,[通訊]網速,不限速卡上傳可,3,台星,21M
3,0,1,114,True,中華電信,,中華、遠傳，蓋一座拆一座,1,中華、遠傳，蓋一座拆一座,中華、遠傳，蓋一座拆一座
4,0,10,674,False,臺灣大哥大,[通訊]國內電信漫遊,越來越爛,1,台灣大哥大,收訊
...,...,...,...,...,...,...,...,...,...,...
1394,6,38,2553,True,遠傳電信,[通訊]網速,遠傳超屌 客戶少了中華3百萬以上 頻寬跟中華只差10M Hz 平均來說 遠傳比中華快多了,5,遠傳超屌 客戶少了中華3百萬以上 頻寬跟中華只差10M Hz 平均來說 遠傳比中華快多了,遠傳超屌 客戶少了中華3百萬以上 頻寬跟中華只差10M Hz 平均來說 遠傳比中華快多了
1395,6,440,23817,True,中華電信,[資費]月租費,中華+台哥大合體 & 在5G非吃到飽部分比較有創意,4,中華+台哥大合體 & 在5G非吃到飽部分比較有創意,中華+台哥大合體 & 在5G非吃到飽部分比較有創意
1396,6,36,2460,False,中華電信,[通訊]網速,很奇怪,2,中華電信,網路
1397,6,99,6196,False,中華電信,[通訊]涵蓋,比較好,4,中華,收訊


In [54]:
aspect_df.batch_idx.value_counts().sort_index()

0     38
1    484
2    258
3    136
4    277
5    142
6     64
Name: batch_idx, dtype: int64

In [16]:
[x for x in range(100) if annots[x]["data"]["thread_idx"] == 16514]

[9]

In [20]:
aa, dbg = cano.process_thread_annotations(annots[9], True)  

In [24]:
len(aspect_list)

521

In [25]:
aa[-1].make_tuple()

('', '', '', -1)